<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare" data-toc-modified-id="Prepare-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare</a></span><ul class="toc-item"><li><span><a href="#Create-Study-Dir" data-toc-modified-id="Create-Study-Dir-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Create Study Dir</a></span></li><li><span><a href="#Select-Cells" data-toc-modified-id="Select-Cells-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Select Cells</a></span></li></ul></li><li><span><a href="#Step-0-Cell-Basic-QC" data-toc-modified-id="Step-0-Cell-Basic-QC-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Step 0 Cell Basic QC</a></span></li><li><span><a href="#Step-1-Prepare-Adata" data-toc-modified-id="Step-1-Prepare-Adata-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Step 1 Prepare Adata</a></span></li><li><span><a href="#Step-2-Dimension-Reduction" data-toc-modified-id="Step-2-Dimension-Reduction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step 2 Dimension Reduction</a></span></li><li><span><a href="#Step-3-Consensus-Clustering" data-toc-modified-id="Step-3-Consensus-Clustering-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step 3 Consensus Clustering</a></span></li><li><span><a href="#Step-4-Marker-Identification" data-toc-modified-id="Step-4-Marker-Identification-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Step 4 Marker Identification</a></span></li><li><span><a href="#Step-5-Cluster-Manual-Annotation" data-toc-modified-id="Step-5-Cluster-Manual-Annotation-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Step 5 Cluster Manual Annotation</a></span></li></ul></div>

In [1]:
from papermill import execute_notebook, PapermillExecutionError
import pandas as pd
import pathlib
import numpy as np

In [2]:
study_name = 'NonN'

In [3]:
total_cell_meta_path = '/home/hanliu/project/mouse_rostral_brain/metadata/CellMeta.110599.Raw.msg'
annotation_table_path = '/home/hanliu/project/mouse_rostral_brain/metadata/CellAnnotation.BrainRegionAndCellClass.msg'
template_dir = '/home/hanliu/project/mouse_rostral_brain/study/ClusteringRecipe/'
study_dir = f'/home/hanliu/project/mouse_rostral_brain/study/Level2-MajorCellType/{study_name}'
dataset_dir = '/home/hanliu/project/mouse_rostral_brain/dataset/'

## Prepare

### Create Study Dir

In [4]:
total_cell_meta = pd.read_msgpack(total_cell_meta_path)
cell_annotation = pd.read_msgpack(annotation_table_path).reindex(total_cell_meta.index)
study_dir = pathlib.Path(study_dir)
study_dir.mkdir(exist_ok=True)
dataset_dir = pathlib.Path(dataset_dir)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Select Cells

In [5]:
selection_dict = {
    'CellClass': [study_name]
}

In [6]:
judges = []
for col, vals in selection_dict.items():
    judges.append(cell_annotation[col].isin(vals))
total_judge = np.all(judges, axis=0)

In [7]:
select_cell_meta = total_cell_meta[total_judge]
cell_meta_path = study_dir / 'CellMetadata.msg'
select_cell_meta.to_msgpack(cell_meta_path)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
print(select_cell_meta.shape)
select_cell_meta.head()

(7441, 22)


,AllcPath,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,Region,...,BamFilteringRate,MappingRate,Pos96,Plate,Col96,Row96,Col384,Row384,FACS_Date,Slice
cell,,,,,,,,,,,,,,,,,,,,,
11E_M_1074,/gale/raidix/rdx-4/mapping/11E/CEMBA190214-11E...,0.003826,0.689524,0.688332,0.006028,0.002211,1345334.0,3039058,2088356.0,11E,...,0.644207,0.687172,D12,CEMBA190214-11E-3,11,3,22,7,190214,11
11E_M_1075,/gale/raidix/rdx-4/mapping/11E/CEMBA190214-11E...,0.004210,0.754025,0.752985,0.011791,0.007613,1539740.0,3470094,2380669.0,11E,...,0.646768,0.686053,D12,CEMBA190214-11E-3,11,3,23,6,190214,11
11E_M_1076,/gale/raidix/rdx-4/mapping/11E/CEMBA190214-11E...,0.004191,0.729799,0.728662,0.011007,0.006844,1572327.0,3629156,2436683.0,11E,...,0.645274,0.671419,D12,CEMBA190214-11E-3,11,3,23,7,190214,11
11E_M_1077,/gale/raidix/rdx-4/mapping/11E/CEMBA190214-11E...,0.004045,0.745161,0.744126,0.011469,0.007454,1377877.0,3245754,2161645.0,11E,...,0.637421,0.665992,D12,CEMBA190214-11E-4,11,3,22,6,190214,11
11E_M_1078,/gale/raidix/rdx-4/mapping/11E/CEMBA190214-11E...,0.004086,0.753881,0.752871,0.011689,0.007634,1090151.0,2644424,1694786.0,11E,...,0.643238,0.640890,D12,CEMBA190214-11E-4,11,3,22,7,190214,11


## Step 0 Cell Basic QC

In [9]:
# 0-CellBasicQC.ipynb
step_0_params = dict(
    study_name=study_name,
    cell_metadata_path=str(cell_meta_path),
)

In [10]:
input_path = '0-CellBasicQC.ipynb'
output_path = study_dir / '0-CellBasicQC.ipynb'
execute_notebook(
    str(input_path),
    str(output_path),
    parameters=step_0_params,
    engine_name=None,
    prepare_only=False,
    kernel_name=None,
    progress_bar=True,
    log_output=False,
    start_timeout=60,
    report_mode=False,
    cwd=str(study_dir))
pass

## Step 1 Prepare Adata

In [11]:
step_1_params = dict(
    # parameters cell
    in_memory=False,
    dask_distribute=True,

    # selected cell metadata path
    cell_meta_path=str(study_dir / 'CellMetadata.AfterQC.msg'),

    # mcds_path
    mcds_path_list=[str(i) for i in dataset_dir.glob('*mcds')],
    clustering_feature='chrom100k',  # usually 100kb chromosome bins or genes

    # remove bad features
    black_list_region=None,
    exclude_chromosome=['chrY', 'chrM'],

    # preprocess parameters
    min_feature_cov=500,
    max_feature_cov=3000,
    ch_hvf_top=3000,
    min_ch_hvf_mean=0.5,
    max_ch_hvf_mean=2.5,
    cg_hvf_top=3000,
    min_cg_hvf_mean=0.5,
    max_cg_hvf_mean=1.2)

In [12]:
input_path = '1-PrepareAdata.ipynb'
output_path = study_dir / '1-PrepareAdata.ipynb'
execute_notebook(
    str(input_path),
    str(output_path),
    parameters=step_1_params,
    engine_name=None,
    prepare_only=False,
    kernel_name=None,
    progress_bar=True,
    log_output=False,
    start_timeout=60,
    report_mode=False,
    cwd=str(study_dir))
pass

## Step 2 Dimension Reduction

In [13]:
step_2_params = dict(
    ch_pc_components=15,
    cg_pc_components=30,
    pc_color_col=None,
    batch_correction_col=None,
    sigma=15,
    alpha=0.1,
    scanorama_k=25,
    n_neighbors=25,
    resolution=0.8,
    perplexity=50,
    metadata_category_cols=['Region'],
    metadata_continue_cols=['CG_RateAdj', 'CH_RateAdj', 'FinalReads'])

In [14]:
input_path = '2-DimensionReduction.ipynb'
output_path = study_dir / '2-DimensionReduction.ipynb'
execute_notebook(
    str(input_path),
    str(output_path),
    parameters=step_2_params,
    engine_name=None,
    prepare_only=False,
    kernel_name=None,
    progress_bar=True,
    log_output=False,
    start_timeout=60,
    report_mode=False,
    cwd=str(study_dir))
pass

## Step 3 Consensus Clustering

In [15]:
step_3_params = dict(
    coord_base='umap',
    n_jobs=40,

    # cc.fit_predict
    n_neighbors=25,
    metric='euclidean',
    neighbor_kwds=None,
    leiden_repeats=200,
    seed=1,
    leiden_resolution=0.3,
    leiden_kwds=None,
    min_cluster_size=10,
    min_cluster_portion=0.01,
    min_samples=1,
    epsilon='auto',
    hdbscan_kwds=None,

    # cc.supervise_training
    x=None,
    test_portion=0.1,
    n_estimators=500,
    n_splits=10,
    fbeta=1,
    average='weighted',
    outlier_proba_cutoff=0.3,
    confusion_merge_cutoff=0.2)

In [16]:
input_path = '3-ConsensusClustering.ipynb'
output_path = study_dir / '3-ConsensusClustering.ipynb'
execute_notebook(
    str(input_path),
    str(output_path),
    parameters=step_3_params,
    engine_name=None,
    prepare_only=False,
    kernel_name=None,
    progress_bar=True,
    log_output=False,
    start_timeout=60,
    report_mode=False,
    cwd=str(study_dir))
pass

## Step 4 Marker Identification

In [17]:
step_4_params = dict(
    mc_type='CGN',
    load=True,

    # find marker gene
    n_marker_genes=1000,
    gene_score_cutoff=5,
    abs_log_fold_change=1,
    n_genes=10,
    nrows=2,
    coord_base='umap')

In [18]:
input_path = '4-MarkerIdentification.ipynb'
output_path = study_dir / '4-MarkerIdentification.ipynb'
execute_notebook(
    str(input_path),
    str(output_path),
    parameters=step_4_params,
    engine_name=None,
    prepare_only=False,
    kernel_name=None,
    progress_bar=True,
    log_output=False,
    start_timeout=60,
    report_mode=False,
    cwd=str(study_dir))
pass

## Step 5 Cluster Manual Annotation
- Prepare Only

In [19]:
step_5_params = dict(load=False, mc_type='CGN', coord_base='umap')

In [20]:
input_path = '5-ClusterManualAnnotation.ipynb'
output_path = study_dir / '5-ClusterManualAnnotation.ipynb'
execute_notebook(
    str(input_path),
    str(output_path),
    parameters=step_5_params,
    engine_name=None,
    prepare_only=True,
    kernel_name=None,
    progress_bar=True,
    log_output=False,
    start_timeout=60,
    report_mode=False,
    cwd=str(study_dir))
pass